# Retrieval augmented chat

This notebook is the primary demonstration of the project. Here we'll bring up the tuned model and vector database and start asking questions both with and without fine tuning or the vector database.

In [26]:
import chromadb
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from IPython.display import Markdown, display

## Initialize some variables

In [7]:
client = chromadb.PersistentClient(path="db/")
collection_name = "Corpus"
merged_model_dir = "merged-fine-tuned"
device_map = {"": 0}
device = "cuda"

## Load the collection

In [3]:
collection = client.get_collection(name = collection_name)

## Load the model

In [8]:
merged_model = AutoModelForCausalLM.from_pretrained(
    merged_model_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

tokenizer = AutoTokenizer.from_pretrained(merged_model_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
def send_to_model(msg):
    messages = [
        {"role": "user", "content": msg},
    ]
    
    encoded = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
    
    generated_ids = merged_model.generate(encoded, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id, temperature=0.8)
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return decoded

def send_chat(msg):
    result = send_to_model(msg)[0]
    return result.rsplit(" [/INST] ", 1)[1]

def basic_chat(msg):
    print(send_chat(msg))

In [ ]:
basic_chat("How do I loop a GIF?")

In [45]:
def printmd(string):
    display(Markdown(string))
    
def retrieval_augmented_chat(msg):
    query_result = collection.query(
        query_texts=[msg], 
        n_results=2
    )
    question_with_context = ""
    if len(query_result['documents'][0]) > 0:
        question_with_context = "Based on the following documents:\n" + "\n\n".join(query_result['documents'][0]) + "\n Answer the following question with lots of details: "
    question_with_context += msg
    model_response = send_chat(question_with_context)

    doc_links = ""
    if len(query_result['metadatas'][0]) > 0:
        doc_links = "\n\n **Reference documents:** \n\n"
        for i in range(0, len(query_result['metadatas'][0])):
            source = query_result['metadatas'][0][i]['source']
            score = query_result['distances'][0][i]
            doc_links += f"* [{source}]({source}) score: {score:3.2f}\n"
    return model_response + doc_links



In [46]:
printmd(retrieval_augmented_chat("How do I loop a GIF?"))

To loop a GIF, you can use a command-line tool called ImageMagick. ImageMagick is an open-source collection of image processing tools that can be used to manipulate and convert images. To loop a GIF using ImageMagick, you can use the `convert` command followed by various options. Here is an example of how you can loop a GIF called `example.gif`:
```bash
convert example.gif -loop 0 example_looped.gif
```
In this example, the `-loop 0` option tells ImageMagick to loop the GIF indefinitely. The output filename `example_looped.gif` is where the looped GIF will be saved.

If you want to loop the GIF for a specific number of times, you can use the `-loop` option followed by the number of times you want the GIF to loop. For example, to loop the GIF 5 times, you would use the following command:
```bash
convert example.gif -loop 5 example_looped.gif
```
In this example, the GIF will loop 5 times before stopping. The output filename `example_looped.gif` is where the looped GIF will be saved.

You can also specify the frame delay time between each loop using the `-delay` option. For example, to loop the GIF with a delay of 100 milliseconds between each loop, you would use the following command:
```bash
convert example.gif -delay 100 -loop 0 example_looped.gif
```
In this example, the GIF will loop indefinitely with a delay of 100 milliseconds between each loop. The output filename `example_looped.gif` is where the looped GIF will be saved.

If you are using macOS, you can install ImageMagick using Homebrew by running the following command in your terminal:
```bash
brew install imagemagick
```
Once ImageMagick is installed, you can use the `convert` command as described above to loop your GIF.

 **Reference documents:** 

* [corpus/imagemagick/set-a-gif-to-loop.md](corpus/imagemagick/set-a-gif-to-loop.md) score: 0.97
* [corpus/imagemagick/compress-animated-gif.md](corpus/imagemagick/compress-animated-gif.md) score: 1.28


In [47]:
printmd(retrieval_augmented_chat("Can I use npx with GitHub actions?"))

Yes, you can use `npx` with GitHub actions. GitHub actions allow you to run scripts with Node.js and install dependencies using `npm`. If you want to use a package installed via `npx`, you can either install it as a dependency in your repository with `npm install [package]`, or you can use `npx` to run the package as a single command.

When using `npx`, you do not need to have a `package.json` file in your repository. Instead, you can specify the package name and version as arguments in your script. For example, if you have a package called `my-package` that you want to install via `npx` and run as a command, you can include the following line in your script:
```javascript
npx my-package [options]
```
Replace `my-package` with the name of your package and `[options]` with any arguments you want to pass to the package. If your package does not have a `bin` field in its `package.json` file, you may need to specify the command you want to run as the first argument.

 **Reference documents:** 

* [corpus/github-actions/npm-cache-with-npx-no-package.md](corpus/github-actions/npm-cache-with-npx-no-package.md) score: 0.88
* [corpus/github-actions/attach-generated-file-to-release.md](corpus/github-actions/attach-generated-file-to-release.md) score: 1.11
